In [1]:
import numpy as np
import data_analysis_codes.ebweyl.ebweyl as ebw
import sympy

In [2]:
L = 2
N = 32 # Number of data points per side
dx = L / N  # Elementary grid size

# Cartesian coordinates
xyz = np.arange(- L / 2, L / 2, dx)
x, y, z = np.meshgrid(xyz, xyz, xyz, indexing='ij')

In [3]:
t = 0.5  # An arbitrary time
B = (x+np.log(t))  # Function from referenced book
Box_0 = np.zeros([N, N, N])
Box_1 = np.ones([N, N, N])

gdown4 = np.array([[-Box_1, Box_0, Box_0, Box_0],
                   [Box_0, (t*t)*Box_1, Box_0, Box_0],
                   [Box_0, Box_0, t*np.exp(x), t*np.exp(x)*B],
                   [Box_0, Box_0, t*np.exp(x)*B, 
                    t*np.exp(x)*(B*B+1)]])
dtB = 1/t  # Time derivative of B function
# Time derivative of metric
dtgdown4 = np.array([[Box_0, Box_0, Box_0, Box_0],
                     [Box_0, 2*t*Box_1, Box_0, Box_0],
                     [Box_0, Box_0, np.exp(x), 
                      np.exp(x)*B + t*np.exp(x)*dtB],
                     [Box_0, Box_0, np.exp(x)*B + t*np.exp(x)*dtB, 
                      np.exp(x)*(B*B+1) + t*np.exp(x)*(2*dtB*B)]])

Kdown4 = (-1/2)*dtgdown4
kappa = 8*np.pi  # Einstein's constant with G = c = 1
Tdown4 = np.zeros([4, 4, N, N, N])

In [4]:
FD = ebw.FiniteDifference(dx, N, periodic_boundary=False, 
                          fd_order6=True)
EBW = ebw.Weyl(FD, gdown4, Kdown4)
Gudd3 = EBW.christoffel_symbol_udd3()
RicciTdown3 = EBW.ricci_tensor_down3(Gudd3)
Endown3 = EBW.eweyl_n_tensor_down3(RicciTdown3, kappa, Tdown4)
Bndown3 = EBW.bweyl_n_tensor_down3(Gudd3)
Cdown4 = EBW.weyl_tensor_down4(Endown3, Bndown3)
Cdown4 = Cdown4[:, :, :, :, int(N/2), int(N/2), int(N/2)]

In [5]:
lup4, kup4, mup4, mbup4 = EBW.null_vector_base(EBW.nup4)
lup4 = lup4[:, int(N/2), int(N/2), int(N/2)]
kup4 = kup4[:, int(N/2), int(N/2), int(N/2)]
mup4 = mup4[:, int(N/2), int(N/2), int(N/2)]
mbup4 = mbup4[:, int(N/2), int(N/2), int(N/2)]
print(lup4)

psi0 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, mup4, lup4, mup4)
psi1 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, kup4, lup4, mup4)
psi2 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, mup4, mbup4, kup4)
psi3 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, kup4, mbup4, kup4)
psi4 = np.einsum('abcd, a, b, c, d -> ', Cdown4, mbup4, kup4, mbup4, kup4)
print('Psi_0 = ', np.real(psi0), ' + i(', np.imag(psi0), ')')
print('Psi_1 = ', np.real(psi1), ' + i(', np.imag(psi1), ')')
print('Psi_2 = ', np.real(psi2), ' + i(', np.imag(psi2), ')')
print('Psi_3 = ', np.real(psi3), ' + i(', np.imag(psi3), ')')
print('Psi_4 = ', np.real(psi4), ' + i(', np.imag(psi4), ')')

[ 0.70710678 -1.41421356 -0.         -0.        ]
Psi_0 =  3.2823288864669564e-08  + i( -4.2316178339163457e-10 )
Psi_1 =  3.8428936114851905e-15  + i( 6.448711543421172e-15 )
Psi_2 =  -3.977536633392731e-09  + i( -8.952345698087782e-09 )
Psi_3 =  -3.837086578295412e-15  + i( 7.068927419883331e-15 )
Psi_4 =  4.000000026851171  + i( -2.259568437912307e-08 )


# Class II transformation

In [6]:
from sympy.abc import k
roots = sympy.solve(psi0 + 4*k*psi1 + 6*(k**2)*psi2 + 4*(k**3)*psi3 + (k**4)*psi4)
b = complex(roots[0])
roots

[-0.0067521620639798 - 0.00670856157570628*I,
 -0.00670828205432095 + 0.00675144248695898*I,
 0.00670828209005178 - 0.00675144250794307*I,
 0.0067521620282528 + 0.00670856159668331*I]

In [7]:
psi0 + 4*b*psi1 + 6*(b**2)*psi2 + 4*(b**3)*psi3 + (b**4)*psi4

(-2.2036853848065186e-16+2.855300554007813e-18j)

In [8]:
lup4 = lup4 + np.conj(b)*mup4 + b*mbup4 + b*np.conj(b)*kup4
mup4 = mup4 + b*kup4
mbup4 = mbup4 + np.conj(b)*kup4

In [9]:
print(lup4)

[ 0.70717084+0.j -1.41408544+0.j -0.02280437+0.j -0.01341712+0.j]


In [10]:
psi0 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, mup4, lup4, mup4)
psi1 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, kup4, lup4, mup4)
psi2 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, mup4, mbup4, kup4)
psi3 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, kup4, mbup4, kup4)
psi4 = np.einsum('abcd, a, b, c, d -> ', Cdown4, mbup4, kup4, mbup4, kup4)
print('Psi_0 = ', np.real(psi0), ' + i(', np.imag(psi0), ')')
print('Psi_1 = ', np.real(psi1), ' + i(', np.imag(psi1), ')')
print('Psi_2 = ', np.real(psi2), ' + i(', np.imag(psi2), ')')
print('Psi_3 = ', np.real(psi3), ' + i(', np.imag(psi3), ')')
print('Psi_4 = ', np.real(psi4), ' + i(', np.imag(psi4), ')')

Psi_0 =  -4.6710680643157704e-17  + i( 2.6963062903479426e-16 )
Psi_1 =  2.4150867284795842e-06  + i( -2.4623249053369562e-06 )
Psi_2 =  2.343601019064936e-06  + i( 0.0003623694098766386 )
Psi_3 =  -0.02700864858881083  + i( -0.02683424633038126 )
Psi_4 =  4.000000026851171  + i( -2.259568437912307e-08 )


# Class I transformation

In [11]:
from sympy.abc import k
roots = sympy.solve(psi3 + 3*k*psi2 + 3*(k**2)*psi1)
ab = complex(roots[0])
a = np.conj(ab)
roots

[15.8900288064012 - 58.5420173388813*I, 58.642123908698 - 15.5120660564864*I]

In [12]:
psi3 + 3*ab*psi2 + 3*(ab**2)*psi1

(-9.71445146547012e-17-4.163336342344337e-17j)

In [13]:
kup4 = kup4 + ab*mup4 + a*mbup4 + a*ab*lup4
mup4 = mup4 + a*lup4
mbup4 = mbup4 + ab*lup4

In [14]:
psi0 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, mup4, lup4, mup4)
psi1 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, kup4, lup4, mup4)
psi2 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, mup4, mbup4, kup4)
psi3 = np.einsum('abcd, a, b, c, d -> ', Cdown4, lup4, kup4, mbup4, kup4)
psi4 = np.einsum('abcd, a, b, c, d -> ', Cdown4, mbup4, kup4, mbup4, kup4)
print('Psi_0 = ', np.real(psi0), ' + i(', np.imag(psi0), ')')
print('Psi_1 = ', np.real(psi1), ' + i(', np.imag(psi1), ')')
print('Psi_2 = ', np.real(psi2), ' + i(', np.imag(psi2), ')')
print('Psi_3 = ', np.real(psi3), ' + i(', np.imag(psi3), ')')
print('Psi_4 = ', np.real(psi4), ' + i(', np.imag(psi4), ')')

Psi_0 =  -2.3701110459056983e-16  + i( -6.693722465863303e-17 )
Psi_1 =  2.4150871991566954e-06  + i( -2.462323202777589e-06 )
Psi_2 =  -0.00020920373843174168  + i( 1.3484835415753916e-06 )
Psi_3 =  1.6593021583750152e-09  + i( -6.183256008326216e-09 )
Psi_4 =  -0.00012408167780578964  + i( 0.7697864041256334 )


# Invariants

In [16]:
psi1*psi1*psi4 + 2*(psi2**3) # if = 0 then the spacetime is special

(-9.154379478069185e-12+1.7822828229701432e-13j)